# SOEN 6611 - Step 5 - Implementation of Measurement Process

### Business Goal
Improve quality of big data for machine learning model.

### Machine Learning Goal
The goal of the machine learning model is to predict whether a particular individual is at high/low credit risk.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data Collection

Out dataset is a credit card dataset containing details of individuals like age, income etc. and a target value stating whether the particular individual is at high or low credit risk. The data is coleected in a form of CSV file and is loaded into the memory using the pandas library. 

#### Assumption
We are considering this file as a single dataset split into three time frames T1, T2 and T3.<br>
$Nds$ = number of datasets = 1

Source: https://www.kaggle.com/datasets/samuelcortinhas/credit-card-classification-clean-data

In [2]:
#import data into dataframe

df = pd.read_csv('data_manipulated.csv')

#split data frame into three parts assuming each part is a separate time frame

df_t1 = df.iloc[:4000]
df_t2 = df.iloc[4000:7000]
df_t3 = df.iloc[7000:]

print('Count:', df_t1.shape[0])
print('Count:', df_t2.shape[0])
print('Count:', df_t3.shape[0])

Count: 4000
Count: 3000
Count: 2828


### Global Variables

num_request - number of requests to database. 
<br>
num_successful_request - number of successful requests to database. <br>
**It is assumed that dataframe is equivalent to database and each call to dataframe is considered as request. If the call does not return any error then num_successful_requests is incremented.**

In [14]:
num_request = 0
num_successful_request = 0

### Base Measure Function


In [10]:
# Method : recCount(df_list) : method to calculate total number of records in multiple datasets <br>
# Param : df_list : list of multiple dataframes 
# Return : Integer : returns total number of records in multiple datasets
def recCount(df_list):
    global num_request
    global num_successful_request
    num_records = 0
    for i in df_list:
        num_request += 1
        num_records += i.shape[0]
        num_successful_request += 1
    
    return num_records

## Big Data Quality Indicators
### Veracity

Veracity refers to the degree that data is accurate, trusted and precise. It is not only the accuracy of the data itself but the trustworthiness of the data source, type, and processing of it.
<br>
$Veracity = W_{acc} * Accuracy + W_{comp} * Completeness + W_{curr} * Currentness + W_{avail} * Availability$

Derived Measures:
- Accuracy
- Completeness
- Currentness
- Availability

#### Accuracy
Degree to which data has attributes that correctly represent the true value of the intended attribute of a concept or event in a specific context of use.
<br><br>
$H_{acc} = log_2(Lbd) - (1/Lbd) * \sum{p_j * log_2(p_j)}$ <br>
$H_{max} = log_2(Lbd)$ <br>
$Accuracy = H_{acc} / H_{max}$ <br>

$Lbd$ : number of records in dataset <br>
$p_j$ : total number of duplicate records in dataset <br>
$H_{acc}$ : entropy of multiple datasets

#### Completeness
Degree to which subject data associated with an entity has values for all expected attributes and related entity instances in a specific context of use.
<br><br>
$Com_m(MDS) = \frac{rec\_no\_null(MDS)}{Lbd(MDS)}$

$Rec\_no\_null (MDS)$ : Frequency of records (in MDS) with no null values<br>
$Lbd$ : number of records in dataset <br>

#### Currentness
Degree to which data has attributes that are of the right age in a specific context of use. 
<br><br>
$Currentness(MDS) = \frac{rec\_acc\_age(MDS)}{Lbd(MDS)}$

$Rec\_acc\_age(MDS)$ : Provides the total number of records with ages that fall within the acceptable range based on the upper and lower quartiles of the Box and Whisker. <br>
$Lbd(MDS)$ : Total Number of records in MDS

#### Availability
Degree to which data has attributes that enable it to be retrieved by authorized users and/or applications in a specific context of use. 
<br><br>
$Availability(MDS) = \frac{N\_succ\_req (MDS)}{N\_req (MDS)} $

$N\_succ\_req (MDS)$ : Number of successful requests (from an API, server, datastore, origins of data, etc) <br>
$N\_req (MDS)$ : Number of requests


In [11]:
#Accuracy
def getAccuracy(df_list):
    num_records = recCount(df_list)
    global num_request
    global num_successful_request
    sum=0
    for i in df_list:
        num_request += 1
        if i.duplicated().sum() != 0:
            sum += (i.duplicated().sum() * np.log2(i.duplicated().sum()))
        num_successful_request += 1
    
    
    Hacc = np.log2(num_records) - (1/num_records) * sum
    Hmax = np.log2(num_records)
    
    return (Hacc / Hmax) 

#Completeness
def getCompleteness(df_list):
    num_records = recCount(df_list)
    global num_request
    global num_successful_request
    rec_null=0
    for i in df_list:
        num_request += 1
        rec_null += i.isnull().any(axis=1).sum()
        num_successful_request += 1
    
    return ((num_records - rec_null) / num_records)
    

#Currentness
def getCurrentness(df_list):
    num_records = recCount(df_list)
    
    #converting string date to pandas Date type
    combine_df = pd.concat([df_t1,df_t2,df_t3])
    combine_df = combine_df.dropna(subset=['Date'])
    combine_df['Date'] = pd.to_datetime(combine_df['Date'], format='%m/%d/%Y')
    combine_df = combine_df.sort_values(by='Date')
    
    #performing box and whisker analysis on Date column
    n = combine_df.shape[0]
    lower = np.ceil((n+1)/4).astype('int64')-1
    higher = np.ceil((3/4) *(n+1)).astype('int64')-1
    date_lower = combine_df.iloc[lower]['Date']
    date_higher = combine_df.iloc[higher]['Date']
    count = combine_df.loc[combine_df['Date'].between(date_lower,date_higher,inclusive='both')].shape[0]
    
    return count / num_records

#Availability
def getAvailability():
    return num_successful_request / num_request

#Veracity
def getVeracity(df_list, w_acc=1/4, w_comp=1/4, w_curr=1/4 ,w_avail = 1/4):
    
    if((w_acc + w_comp + w_curr + w_avail) > 1):
        print("Sum of weights is greater than one")
        return
    
    return (w_acc * getAccuracy(df_list)) + (w_comp * getCompleteness(df_list)) + (w_curr * getCurrentness(df_list)) + (w_avail * getAvailability())

### Vincularity

### Validity

In [16]:
# Compliance
def getCompliance(df_list):
    # total number of datasets
    nds_df_list = len(df_list)
    sum_mds_comp = 0
    
    for i in df_list:
        num_comp_rec = i['PCI_compliant'].value_counts()['Yes']
        total_rec = i.shape[0]
        i_comp = num_comp_rec/total_rec
        sum_mds_comp += i_comp
    
    mds_compliance = sum_mds_comp/nds_df_list
    print("Compliance: ", mds_compliance)
    return mds_compliance

# Credability
def getCredability(df_list):
    # total number of datasets
    nds_df_list = len(df_list)
    num_cred_dataset = 0
    
    for i in df_list:
        num_cred_dataset += 1
    
    mds_credability = num_cred_dataset/num_cred_dataset
    print("Credability: ", mds_credability)
    return mds_credability

# Validity
def getValidity(credability, cred_weight, compliance, comp_weight):
    ver = credability*cred_weight + compliance*comp_weight
    return ver

In [17]:
# Validity for Time frame 1
print("Validity for Time frame 1: ", getValidity(getCompliance([df_t1]), 0.5, getCredability([df_t1]), 0.5))

Compliance:  0.92125
Credability:  1.0
Validity for Time frame 1:  0.9606250000000001


In [18]:
# Validity for Time frame 2
print("Validity for Time frame 2: ", getValidity(getCompliance([df_t2]), 0.5, getCredability([df_t2]), 0.5))

Compliance:  0.9163333333333333
Credability:  1.0
Validity for Time frame 2:  0.9581666666666666


In [19]:
# Validity for Time frame 3
print("Validity for Time frame 3: ", getValidity(getCompliance([df_t3]), 0.5, getCredability([df_t3]), 0.5))

Compliance:  0.923974540311174
Credability:  1.0
Validity for Time frame 3:  0.9619872701555869


### Measuring Indicators before processing

In [20]:
#Veracity
accuracy = getAccuracy([df_t1,df_t2,df_t3])
completeness = getCompleteness([df_t1,df_t2,df_t3])
currentness = getCurrentness([df_t1,df_t2,df_t3])
availability = getAvailability()
veracity = getVeracity([df_t1,df_t2,df_t3])

print('Accuracy:',accuracy)
print('Completeness:',completeness)
print('Currentness:',currentness)
print('Availability:',availability)
print('Veracity:',veracity)

ValueError: time data '10-11-2012' does not match format '%m/%d/%Y' (match)

In [ ]:
#Vincularity

In [21]:
# Compliance
def getCompliance(df_list):
    # total number of datasets
    nds_df_list = len(df_list)
    sum_mds_comp = 0
    
    for i in df_list:
        num_comp_rec = i['PCI_compliant'].value_counts()['Yes']
        total_rec = i.shape[0]
        i_comp = num_comp_rec/total_rec
        sum_mds_comp += i_comp
    
    mds_compliance = sum_mds_comp/nds_df_list
    print("Compliance: ", mds_compliance)
    return mds_compliance

# Credability
def getCredability(df_list):
    # total number of datasets
    nds_df_list = len(df_list)
    num_cred_dataset = 0
    
    for i in df_list:
        num_cred_dataset += 1
    
    mds_credability = num_cred_dataset/num_cred_dataset
    print("Credability: ", mds_credability)
    return mds_credability

# Validity
def getValidity(credability, cred_weight, compliance, comp_weight):
    ver = credability*cred_weight + compliance*comp_weight
    return ver

In [22]:
# Validity for Time frame 1
print("Validity for Time frame 1: ", getValidity(getCompliance([df_t1]), 0.5, getCredability([df_t1]), 0.5))

# Validity for Time frame 2
print("Validity for Time frame 2: ", getValidity(getCompliance([df_t2]), 0.5, getCredability([df_t2]), 0.5))

# Validity for Time frame 3
print("Validity for Time frame 3: ", getValidity(getCompliance([df_t3]), 0.5, getCredability([df_t3]), 0.5))

Compliance:  0.92125
Credability:  1.0
Validity for Time frame 1:  0.9606250000000001
Compliance:  0.9163333333333333
Credability:  1.0
Validity for Time frame 2:  0.9581666666666666
Compliance:  0.923974540311174
Credability:  1.0
Validity for Time frame 3:  0.9619872701555869


### Analyzing Data

In [25]:
# checking for null values
df.isna().sum()

ID                  0
Gender             16
Own_car            16
Own_property       16
Work_phone         44
Phone              20
Email              16
Unemployed         16
Num_children       16
Num_family         44
Account_length     16
Total_income       16
Age                16
Years_employed     16
Income_type        16
Education_type     16
Family_status      16
Housing_type       16
Occupation_type    16
Target             16
Date               16
dtype: int64

There are null values in all the columns except ID and these rows with null values needs to be removed as part of cleaning the dataset.

In [68]:
#checking duplicate records
df.duplicated().sum()

11

There are 11 duplicate rows which needs to be removed.

In [79]:
# checking for gender, own_car, own_property, work_phone, phone, email, unemployed and Target columns not having values 1 or 0
count_gender = (~df['Gender'].isna() & ~df['Gender'].isin([1,0])).sum()
count_car = (~df['Own_car'].isna() & ~df['Own_car'].isin([1,0])).sum()
count_property = (~df['Own_property'].isna() & ~df['Own_property'].isin([1,0])).sum()
count_work_phone = (~df['Work_phone'].isna() & ~df['Work_phone'].isin([1,0])).sum()
count_phone = (~df['Phone'].isna() & ~df['Phone'].isin([1,0])).sum()
count_email = (~df['Email'].isna() & ~df['Email'].isin([1,0])).sum()
count_unemployed = (~df['Unemployed'].isna() & ~df['Unemployed'].isin(['1','0'])).sum()
count_target = (~df['Target'].isna() & ~df['Target'].isin([1,0])).sum()

print('Non Numeric Values Gender:', count_gender)
print('Non Numeric Values Car:', count_car)
print('Non Numeric Values Property:', count_property)
print('Non Numeric Values Work Phone:', count_work_phone)
print('Non Numeric Values Phone:', count_phone)
print('Non Numeric Values Email:', count_email)
print('Non Numeric Values Unemployed:', count_unemployed)
print('Non Numeric Values Target:', count_target)

Non Numeric Values Gender: 0
Non Numeric Values Car: 0
Non Numeric Values Property: 0
Non Numeric Values Work Phone: 0
Non Numeric Values Phone: 32
Non Numeric Values Email: 0
Non Numeric Values Unemployed: 56
Non Numeric Values Target: 52


### Data Processing

In [23]:
def preprocess(df):
    
    #remove rows with atleast 1 null value
    df = df.dropna()
    
    return df

#Function call
df_t1_processed = preprocess(df_t1)
df_t2_processed = preprocess(df_t2)
df_t3_processed = preprocess(df_t3)

print('Count:', df_t1_processed.shape[0])
print('Count:', df_t2_processed.shape[0])
print('Count:', df_t3_processed.shape[0])

Count: 3966
Count: 2966
Count: 2828


### Measuring Indicators after processing

### Visualizing and Interpreting Indicators